## Project: Traffic signs recognition and

We'll check out how to build a **convolutional network** to classify CIFAR10 images. By using weight sharing - multiple units with the same weights - convolutional layers are able to learn repeated patterns in your data. For example, a unit could learn the pattern for an eye, or a face, or lower level features like edges.


In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import time

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as utils
from readTrafficSigns import readTrafficSigns
import random
import math
import scipy.misc 

In [2]:
def get_data_TrafficSigns():
    """
    Load the German Traffic Sign Recognition Benchmark dataset from disk and perform preprocessing to prepare
    it for the linear classifier.  
    """
    # Load the raw traffic signs data
    trafficSigns_dir = './data/train/Final_Training/Images'
    X_train, y_train = readTrafficSigns(trafficSigns_dir)
    
    
    #print (len(X_train))   #vector of dimension 39209 --> one element per image : for each image (element) we get a matrix for each pixel containing the RGBs
    #print (len(y_train))   #vector of dimension 39209 --> one element per image : for each image (element) we get the number of the class it belongs to 
    #print (y_train[0])
    #print (type(y_train))
     #for the moment, images are classed in order --> firt 210 images belong to class 0 (y=0), next ones belong to class 1 etc...

    ind = list()
    
    # subsample the data
    for i in range(0,43):
        
        index_value = list()
        for j in range (0,39209):
            if int(y_train[j]) == i:
                index_value.append(j)
        ind.append(random.sample(range(min(index_value),max(index_value)+1), math.floor(0.02*len(index_value))))
    
    ind2 = [item for sublist in ind for item in sublist]
    X_val = [X_train[i] for i in ind2]
    y_val = [y_train[i] for i in ind2]
    for ind2 in sorted(ind2, reverse=True):
        del X_train[ind2]
        del y_train[ind2]
   
    return X_train, y_train, X_val, y_val

# Invoke the above function to get our data.
X_train, y_train, X_val, y_val = get_data_TrafficSigns()
 
 

In [4]:
def resize(X, length, width):
    X_new = np.empty(shape=(0,3,length,width))
    for i in range(len(X)):
        image = X[i];
        image = scipy.misc.imresize(image, (length, width, 3))
        transp_image = image.transpose(2,0,1)
        y = np.expand_dims(transp_image, axis=0)
        X_new = np.concatenate((X_new,y), axis=0)

    return X_new  

n = 47
X_train = resize(X_train, n,n)
X_val = resize(X_val, n,n)
y_train = np.asarray(y_train)
y_val = np.asarray(y_val)

print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)

X_train, y_train = torch.from_numpy(X_train).type(torch.FloatTensor), torch.from_numpy(y_train).type(torch.LongTensor)
X_val, y_val = torch.from_numpy(X_val).type(torch.FloatTensor), torch.from_numpy(y_val).type(torch.LongTensor)

traindataset = utils.TensorDataset(X_train, y_train)
trainloader = utils.DataLoader(traindataset, batch_size=64, shuffle=True)

valdataset = utils.TensorDataset(X_val, y_val)
valloader = utils.DataLoader(valdataset, batch_size=64, shuffle=True)
    

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  """


KeyboardInterrupt: 

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, n_input_channels=3, n_output=43):
        super().__init__()
        ################################################################################
        # TODO:                                                                        #
        # Define 2 or more different layers of the neural network                      #
        ################################################################################
        
        self.conv1 = nn.Conv2d(n_input_channels, 70, 5, stride = 1, padding = 2)
        self.pool = nn.MaxPool2d(3, stride=2)
        self.conv2 = nn.Conv2d(70, 110, 3, stride = 1, padding = 1)
        self.conv3 = nn.Conv2d(110,180,3, stride=1, padding = 1)
        self.fc1 = nn.Linear(180 * 5 * 5, 200)
        self.fc2 = nn.Linear(200, 100)
        self.fc3 = nn.Linear(100, n_output)
        
        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################
    
    def forward(self, x):
        ################################################################################
        # TODO:                                                                        #
        # Set up the forward pass that the input data will go through.                 #
        # A good activation function betweent the layers is a ReLu function.           #
        #                                                                              #
        # Note that the output of the last convolution layer should be flattened       #
        # before being inputted to the fully connected layer. We can flatten           #
        # Variable `x` with `x.view`.                                                  #
        ################################################################################  
        
        x = F.relu(self.pool(self.conv1(x)))
        x = F.relu(self.pool(self.conv2(x)))
        x = F.relu(self.pool(self.conv3(x)))
        x = x.view(-1, 180 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################
        
        return x
    
    def predict(self, x):
        logits = self.forward(x)
        return F.softmax(logits)

In [ ]:
net = ConvNet()
################################################################################
# TODO:                                                                        #
# Choose an Optimizer that will be used to minimize the loss function.         #
# Choose a critera that measures the loss                                      #
################################################################################

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.005)

epochs = 3
steps = 0
running_loss = 0
print_every = 30
for e in range(epochs):
    start = time.time()
    for images, labels in iter(trainloader):
        
        steps += 1
        ################################################################################
        # TODO:                                                                        #
        # Run the training process                                                     #
        #                                                                              #
        # HINT: Do not forget to transform the inputs and outputs into Variable        #
        # which pytorch uses.                                                          #
        ################################################################################
        
        #transofrm inputs and outputs into Variable 
        inputs, targets = Variable(images), Variable(labels)
        
        #set gradient to zero
        optimizer.zero_grad()
        
        # forward pass
        out = net.forward(inputs)
        
        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################
        
        loss = criterion(out, targets)
        
        ################################################################################
        # TODO:                                                                        #
        # Run the training process                                                     #
        #                                                                              #
        # HINT: Calculate the gradient and move one step further                       #
        ################################################################################
        
        loss.backward()
        optimizer.step()
        
        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################
        
        running_loss += loss.data[0]
        
        if steps % print_every == 0:
            stop = time.time()
            # Test accuracy
            accuracy = 0
            for ii, (images, labels) in enumerate(valloader):
                
                ################################################################################
                # TODO:                                                                        #
                # Calculate the accuracy                                                       #
                ################################################################################
                
                out = net.predict(Variable(images))
                _, prediction = torch.max(out, 1)
                pred_y = prediction.data.numpy().squeeze()
                target_y = (labels.numpy()).data
                accuracy += sum(pred_y == target_y)/64
                
                ################################################################################
                #                              END OF YOUR CODE                                #
                ################################################################################
            
            print("Epoch: {}/{}..".format(e+1, epochs),
                  "Loss: {:.4f}..".format(running_loss/print_every),
                  "Test accuracy: {:.4f}..".format(accuracy/(ii+1)),
                  "{:.4f} s/batch".format((stop - start)/print_every)
                 )
            running_loss = 0
            start = time.time()

In [ ]:
torch.save(net.state_dict(), 'project.pt')